**Classfication of Cats and Dogs with Keras (Tensoflow backend)**

Here we shall use, the data set provided by [Kaggle](https://www.kaggle.com/c/dogs-vs-cats). I have segregated the given dataset into a structure containing different sets for training and validation.

Directory structure:

dataset/

    training_set/
    
        cats/
        
        dogs/
        
    test_set/
    
        cats/
        
        dogs/
        
    This notebook generates a CNN model for image recognition task and saves it.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense, Dropout
#print(check_output(["ls", "../input/dataset/dataset/training_set/cats"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

First, lets declare the parameters width and height of the image we would like to transform our images to. 
I have taken 60 as it generates less number of units in convolution layers and hence requires less time to fit model.

In [ ]:
shape_w = 60
shape_h = 60
batchsize = 32

Now, declare a sequential Neural network and add three convolution layers to it.
Here I have used 32 filters each 3*3 matrix in the first 2 layers and 64 filters in the last conv layer.

In [ ]:
classifier = Sequential()

classifier.add(Conv2D(32, (3, 3), activation="relu", input_shape=(shape_w, shape_h, 3)))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

classifier.add(Conv2D(32, (3, 3), activation="relu"))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

classifier.add(Conv2D(64, (3, 3), activation="relu"))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

Now, we have to flatten the matrices to a single dimensional vector for the fully connected network

In [ ]:
classifier.add(Flatten())

Adding a fully connected network with 1 hidden layer and dropout of 0.5, which tries to prevent model-overfitting.

In [ ]:
classifier.add(Dense(units=128, activation="relu"))
classifier.add(Dropout(0.5))
classifier.add(Dense(units=1, activation="sigmoid"))

Compile the model with 'adam' optimizer'

In [ ]:
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])


**Generate image data from ImageDataGenerator**
This helps in training the model on more images from a small set of training images. This can be used to prevent model-overfitting by generating various images with transformations.

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

validation_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('../input/dataset/dataset/training_set',
                                                 target_size = (shape_w, shape_h),
                                                 batch_size = batchsize,
                                                 class_mode = 'binary')

validation_set = validation_datagen.flow_from_directory('../input/dataset/dataset/test_set',
                                            target_size = (shape_w, shape_h),
                                            batch_size = batchsize,
                                            class_mode = 'binary')

**Fit the model and save**

In [ ]:
classifier.fit_generator(training_set,
                         steps_per_epoch = (10000 // batchsize),
                         epochs = 50,
                         validation_data = validation_set,
                         validation_steps = (2500 // batchsize))

classifier.save('catsdogs.h5')
classifier.save_weights('catsdogs_weights.h5')